<a href="https://colab.research.google.com/github/Rishabh181/dataextraction_dataanalysi/blob/main/Blackcofferassignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install selenium

In [9]:
import os
import re
import nltk
import pandas as pd

# Updated imports for Chrome
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium import webdriver
#from selenium.webdriver import Chrome  # Use 'webdriver' for Chrome

from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from collections import Counter
from string import punctuation

In [10]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
# Paths
STOPWORDS_PATH = "Blackcoffer/StopWords/"
MASTER_DICT_PATH = "Blackcoffer/MasterDictionary/"

In [12]:
# Load Stop Words
def load_stop_words():
    stop_words = set()
    for file_name in os.listdir(STOPWORDS_PATH):
        try:
            with open(os.path.join(STOPWORDS_PATH, file_name), 'r', encoding='utf-8') as f:
                stop_words.update(f.read().splitlines())
        except UnicodeDecodeError:
            try:
                with open(os.path.join(STOPWORDS_PATH, file_name), 'r', encoding='latin-1') as f:  # Or 'cp1252'
                    stop_words.update(f.read().splitlines())
            except UnicodeDecodeError:
                print(f"Warning: Could not decode file {file_name} with either UTF-8 or Latin-1 encoding.")
    return stop_words

In [13]:

# # Load Positive and Negative Words
# def load_master_dict():
#     positive_words = set()
#     negative_words = set()
#     with open(os.path.join(MASTER_DICT_PATH, "positive-words.txt"), 'r', encoding='latin-1') as f:
#         positive_words.update(f.read().splitlines())
#     with open(os.path.join(MASTER_DICT_PATH, "negative-words.txt"), 'r',  encoding='latin-1') as f:
#         negative_words.update(f.read().splitlines())
#     return positive_words, negative_words

def load_master_dict():
    positive_words = set()
    negative_words = set()

    for file_path, word_set in [(os.path.join(MASTER_DICT_PATH, "positive-words.txt"), positive_words),
                               (os.path.join(MASTER_DICT_PATH, "negative-words.txt"), negative_words)]:
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                word_set.update(f.read().splitlines())
        except UnicodeDecodeError:
            try:
                with open(file_path, 'r', encoding='latin-1') as f:
                    word_set.update(f.read().splitlines())
            except UnicodeDecodeError:
                print(f"Warning: Could not decode file {file_path} with either UTF-8 or Latin-1 encoding.")

    return positive_words, negative_words

In [14]:
STOP_WORDS = load_stop_words()
POSITIVE_WORDS, NEGATIVE_WORDS = load_master_dict()

FileNotFoundError: [Errno 2] No such file or directory: 'Blackcoffer/StopWords/'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
f = open(os.path.join(MASTER_DICT_PATH, "negative-words.txt"), 'r',  encoding='latin-1')
print(f.read())

In [ ]:
# Clean and tokenize text
def clean_text(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text.lower())
    return [word for word in tokens if word not in STOP_WORDS and word not in punctuation]

In [ ]:
# Calculate variables
def calculate_scores(tokens, text):
    positive_score = sum(1 for word in tokens if word in POSITIVE_WORDS)
    negative_score = sum(1 for word in tokens if word in NEGATIVE_WORDS)
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(tokens) + 0.000001)

    sentences = sent_tokenize(text)
    avg_sentence_length = len(tokens) / len(sentences)

    complex_words = [word for word in tokens if count_syllables(word) > 2]
    percentage_complex_words = len(complex_words) / len(tokens)
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

    avg_words_per_sentence = len(tokens) / len(sentences)
    syllables_per_word = sum(count_syllables(word) for word in tokens) / len(tokens)

    personal_pronouns = len(re.findall(r"\b(I|we|my|ours|us)\b", text, re.IGNORECASE))
    avg_word_length = sum(len(word) for word in tokens) / len(tokens)

    return {
        "Positive Score": positive_score,
        "Negative Score": negative_score,
        "Polarity Score": polarity_score,
        "Subjectivity Score": subjectivity_score,
        "Avg Sentence Length": avg_sentence_length,
        "Percentage of Complex Words": percentage_complex_words,
        "Fog Index": fog_index,
        "Avg Number of Words Per Sentence": avg_words_per_sentence,
        "Complex Word Count": len(complex_words),
        "Word Count": len(tokens),
        "Syllable Per Word": syllables_per_word,
        "Personal Pronouns": personal_pronouns,
        "Avg Word Length": avg_word_length,
    }

In [ ]:
# Count syllables in a word
def count_syllables(word):
    word = word.lower()
    vowels = "aeiou"
    count = sum(1 for char in word if char in vowels)
    if word.endswith("es") or word.endswith("ed"):
        count -= 1
    return max(1, count)


In [ ]:
# Extract article text using Selenium
def extract_text_from_url(url):
    options = EdgeOptions()
    options.add_argument("--headless")  # Run Edge in headless mode
    options.use_chromium = True
    service = EdgeService('path_to_msedge_driver')  # Provide the full path to your Edge WebDriver
    driver = Edge(service=service, options=options)

    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        title = soup.find('h1').get_text(strip=True)
        paragraphs = soup.find_all('p')
        article_text = ' '.join(p.get_text(strip=True) for p in paragraphs)
        return title, article_text
    finally:
        driver.quit()

In [ ]:
# Main Function
def main():
    # Load input data
    input_df = pd.read_excel("Blackcoffer/Input.xlsx")

    output_data = []
    for index, row in input_df.iterrows():
        url_id = row['URL_ID']
        url = row['URL']

        try:
            title, article_text = extract_text_from_url(url)
            tokens = clean_text(article_text)
            scores = calculate_scores(tokens, article_text)

            output_row = {
                "URL_ID": url_id,
                "URL": url,
                **scores
            }
            output_data.append(output_row)

            # Save article text to file
            os.makedirs("articles", exist_ok=True)
            with open(f"articles/{url_id}.txt", "w", encoding="utf-8") as f:
                f.write(title + "\n" + article_text)

        except Exception as e:
            print(f"Error processing URL_ID {url_id}: {e}")

    # Save output data
    output_df = pd.DataFrame(output_data)
    output_df.to_excel("Output Data Structure.xlsx", index=False)

if __name__ == "__main__":
    main()

In [ ]:

# if __name__ == "__main__":
#     main()
